### Structured Output

Models can be requested to provide their response in a format matching a given schema.
This is useful for ensuring the output can be easily parsed and used in subsequent processing.
LangChain supports multiple schema types and methods for enforcing structured output.

#### Using Pydantic

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain.chat_models import init_chat_model

model = init_chat_model(model="groq:llama-3.1-8b-instant")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000027D7E387490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027D7E447150>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
# defining schema for structured output
from pydantic import BaseModel, Field
class State(BaseModel):
    movie_title: str=Field(description="this is the title of the movie")
    main_actor: str=Field(description="Main protagonist in movie")
    genre: str=Field(description="this is the genre of the movie")
    ratings: float=Field(description="imdb ratings of the movie")

structured_model = model.with_structured_output(State) 

structured_model

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000027D7E387490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027D7E447150>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'State', 'description': '', 'parameters': {'properties': {'movie_title': {'description': 'this is the title of the movie', 'type': 'string'}, 'main_actor': {'description': 'Main protagonist in movie', 'type': 'string'}, 'genre': {'description': 'this is the genre of the movie', 'type': 'string'}, 'ratings': {'description': 'imdb ratings of the movie', 'type': 'number'}}, 'required': 

In [3]:
#model response without schema

model.invoke("Say about movie Bahubaali")

AIMessage(content="Bahubali is a two-part Indian action film written and directed by SS Rajamouli. The film is based on the ancient Indian epic, Kishkindha, from the Hindu mythology. The story revolves around Amarendra Baahubali (played by Prabhas) and his journey to become the king of Mahishmati.\n\n**Storyline**\n\nThe film is divided into two parts: Bahubali 1 and Bahubali 2. Here's a brief summary of each part:\n\n**Bahubali 1: The Beginning**\n\nThe film begins with the story of Amarendra Baahubali, a brave and kind-hearted prince who is adopted by the king of Mahishmati. Baahubali is trained by a wise old man named Kattappa (played by Sathyaraj) and becomes the perfect warrior. He falls in love with Devasena (played by Anushka Shetty) and marries her. However, their happiness is short-lived, as Baahubali is betrayed by Kattappa, who kills him on the orders of the king's wife, Sivagami (played by Ramya Krishnan).\n\n**Bahubali 2: The Conclusion**\n\nThe second part of the film tak

In [4]:
#model response with structured schema

structured_model.invoke("Say about movie Bahubaali")

State(movie_title='Bahubaali', main_actor='Prabhas', genre='Action', ratings=8.6)

In [7]:
# we can also include raw response

with_raw_response = model.with_structured_output(schema=State, include_raw=True)
with_raw_response.invoke("Say about movie Bahubaali")

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'rcbh4v8mc', 'function': {'arguments': '{"genre":"Action","main_actor":"Prabhas","movie_title":"Bahubali","ratings":9.2}', 'name': 'State'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 283, 'total_tokens': 319, 'completion_time': 0.048234249, 'completion_tokens_details': None, 'prompt_time': 0.020929283, 'prompt_tokens_details': None, 'queue_time': 0.055729326, 'total_time': 0.069163532}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b420d-0f3c-7e83-bc2b-ae02aecdabf6-0', tool_calls=[{'name': 'State', 'args': {'genre': 'Action', 'main_actor': 'Prabhas', 'movie_title': 'Bahubali', 'ratings': 9.2}, 'id': 'rcbh4v8mc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 283, 'output_tokens': 36, 'total_tokens': 319

In [8]:
#nested structure example
from typing import List

class Actor(BaseModel):
    name: str=Field(description="name of the actor")
    role: str=Field(description="role played in movie")

class Movie(BaseModel):
    title: str=Field(description="Ttile of the movie")
    actor: List[Actor]  
    rating: float=Field(description="imdb rating of the movie") 

In [10]:
#nested schema model

nested_schema_model = model.with_structured_output(schema=Movie)
nested_schema_model.invoke("say about movie titanic")

Movie(title='Titanic', actor=[Actor(name='Leonardo DiCaprio', role='Jack Dawson'), Actor(name='Kate Winslet', role='Rose DeWitt Bukater')], rating=7.5)

#### Using TypedDict

In [11]:
from typing_extensions import TypedDict
from typing import Annotated

class MovieDict(TypedDict):
    "a movie with details"
    name: Annotated[str, ..., "the title of the movie"]
    director: Annotated[str, ..., "the director of the movie"]
    year: Annotated[int, ..., "the year in which movie got released"]


model_with_dict = model.with_structured_output(MovieDict)
model_with_dict.invoke("say about titanic movie")    

{'director': 'James Cameron', 'name': 'Titanic', 'year': 1997}

In [13]:
#profile attribute is only available for base model with no output schema defined
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 8192,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': False,
 'tool_calling': True}

#### Using Dataclasses

In [ ]:
from dataclasses import dataclass



